This notebook will pull from the FPL API the weekly results for each player and stack all the results into a single df

Update weekly

In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
top500 = pd.read_csv('top500.csv', index_col='id_')
teams = pd.read_csv('teams', index_col=0)

In [3]:
histories = {}

for id_ in top500.index:
    url = 'https://fantasy.premierleague.com/api/element-summary/' + str(id_) + '/'
    r = requests.get(url)
    player_data = r.json()
    player_history = player_data['history']
    
    histories[id_] = player_history

In [4]:
salah = pd.DataFrame(histories[191])

In [5]:
salah['total_points']

0     12
1      3
2     15
3      6
4      9
5      5
6      3
7      2
8      0
9      7
10     2
11     8
12     0
13     3
14     0
15    13
16    16
17     3
18     3
19    10
20     6
21     7
22     2
23    14
Name: total_points, dtype: int64

In [6]:
history_dfs = {}

for key in histories:
    df = pd.DataFrame(histories[key])
    df['id_'] = key
    df['web_name'] = top500.loc[key]['web_name']
    df['team'] = top500['team'].loc[key]
    df['sum_minutes'] = [sum(df['minutes'].iloc[:i+1]) for i in range(len(df.index))]
    df['sum_points'] = [sum(df['total_points'].iloc[:i+1]) for i in range(len(df.index))]
    df['next_week_points'] = df['total_points'].drop(0).append(pd.Series(0)).reset_index(drop=True)
    
    history_dfs[key] = df

In [7]:
pd.set_option('display.max_columns', None)
history_dfs[191]

,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,id_,web_name,team,sum_minutes,sum_points,next_week_points
0,1,2,40,0,39.7,191,1,1,1,13.4,54.8,2019-08-09T19:00:00Z,90,14,0,0,0,0,1,0,2375465,1,4,39.0,12,0,0,0,125,True,0,191,Salah,10,90,12,3
1,0,0,0,1,11.8,191,19,0,0,4.3,2.4,2019-08-17T14:00:00Z,78,16,0,0,0,0,2,0,2778082,2,1,29.0,3,114282,156910,42628,125,False,0,191,Salah,10,168,15,15
2,0,3,43,0,16.6,191,24,1,2,15.5,71.2,2019-08-24T16:30:00Z,90,1,0,0,0,0,3,0,2444065,1,3,67.0,15,-351541,47097,398638,125,True,0,191,Salah,10,258,30,6
3,1,0,15,1,38.4,191,32,0,0,12.3,27.0,2019-08-31T16:30:00Z,90,5,0,0,0,0,4,0,2810462,3,0,58.0,6,243530,291554,48024,125,False,0,191,Salah,10,348,36,9
4,0,2,34,0,41.8,191,44,1,1,15.9,44.6,2019-09-14T11:30:00Z,90,13,0,0,0,0,5,0,2874525,1,3,73.0,9,31898,124574,92676,125,True,0,191,Salah,10,438,45,5
5,1,0,11,0,15.3,191,53,1,0,6.6,14.8,2019-09-22T15:30:00Z,90,6,0,0,0,0,6,0,2804643,2,1,36.0,5,-80384,49872,130256,125,False,0,191,Salah,10,528,50,3
6,0,0,2,1,13.3,191,68,0,0,8.9,7.6,2019-09-28T11:30:00Z,90,15,0,0,0,0,7,0,2889458,1,0,68.0,3,48879,123616,74737,125,False,0,191,Salah,10,618,53,2
7,0,0,5,0,37.7,191,74,1,0,15.0,18.2,2019-10-05T14:00:00Z,90,9,0,0,0,0,8,0,2797360,1,2,94.0,2,-92287,34441,126728,125,True,0,191,Salah,10,708,55,0
8,0,0,0,0,0.0,191,87,0,0,0.0,0.0,2019-10-20T15:30:00Z,0,12,0,0,0,0,9,0,2186184,1,1,0.0,0,-601424,13726,615150,124,False,0,191,Salah,10,708,55,7
9,0,0,25,0,15.3,191,94,1,1,15.3,45.0,2019-10-27T16:30:00Z,84,17,0,0,0,0,10,0,2096834,1,2,93.0,7,-114968,23153,138121,124,True,0,191,Salah,10,792,62,2


In [8]:
top500histories = pd.concat(history_dfs.values(), ignore_index=True)
top500histories.shape

(10495, 37)

In [9]:
top500histories.to_csv('pre_top500histories.csv')

In [10]:
top500histories.columns

Index(['assists', 'bonus', 'bps', 'clean_sheets', 'creativity', 'element',
       'fixture', 'goals_conceded', 'goals_scored', 'ict_index', 'influence',
       'kickoff_time', 'minutes', 'opponent_team', 'own_goals',
       'penalties_missed', 'penalties_saved', 'red_cards', 'round', 'saves',
       'selected', 'team_a_score', 'team_h_score', 'threat', 'total_points',
       'transfers_balance', 'transfers_in', 'transfers_out', 'value',
       'was_home', 'yellow_cards', 'id_', 'web_name', 'team', 'sum_minutes',
       'sum_points', 'next_week_points'],
      dtype='object')

In [11]:
top500histories.shape[0]

10495